In [28]:
%pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.9.1
Uninstalling torch-2.9.1:
  Successfully uninstalled torch-2.9.1
Found existing installation: torchvision 0.24.1
Uninstalling torchvision-0.24.1:
  Successfully uninstalled torchvision-0.24.1
Found existing installation: torchaudio 2.9.1
Uninstalling torchaudio-2.9.1:
  Successfully uninstalled torchaudio-2.9.1
Note: you may need to restart the kernel to use updated packages.


In [29]:
%pip install "torch==2.10.0" "torchvision==0.25.0" "torchaudio==2.10.0"

  Using cached torch-2.10.0-2-cp311-none-macosx_11_0_arm64.whl.metadata (31 kB)
  Using cached torchvision-0.25.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached torchaudio-2.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.9 kB)
Using cached torch-2.10.0-2-cp311-none-macosx_11_0_arm64.whl (79.4 MB)
Using cached torchvision-0.25.0-cp311-cp311-macosx_11_0_arm64.whl (1.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.4/736.4 kB 11.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [torchaudio] 1/3 [torchvision]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.6 requires torch<2.10,>=1.10, but you have torch 2.10.0 which is incompatible.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
import torch

print("Torch version:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Torch version: 2.10.0
MPS available: True
MPS built: True
Using mps device


In [21]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [3]:
test_df = pd.read_json('test.json', lines=True)
train_df = pd.read_json('train.json', lines=True)
test_df

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91
...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91
3630,3630,id_ff9bf3581,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91
3631,3631,id_ffc8f96a8,GGAAAGAUGUUCUGAUGAACAUCGGCUGUUCUAGCUUUCAUCUAUC...,.....(((((((....)))))))(((.(((((((((((((((((((...,EEEEESSSSSSSHHHHSSSSSSSSSSBSSSSSSSSSSSSSSSSSSS...,130,91
3632,3632,id_ffd7e8cc1,GGAAACCGUUAACCUGCAUCUUCAUGUUAUCGCUUGCGACAGCAAC...,...............................((((((............,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSIIIIIIIII...,130,91


In [36]:
train_df

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693, 1.1227, 0.8686, 0.7217, 0.438...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021000000000001, 0.9804, 0.4967000...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7975, 1.4991, 0.8686, 0..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818000000000002, 1.9919, 2.5485, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437, 1.0932, 0.0, 0.0, 1.13...","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947, 0.7457, 3.1233, 0.8086, 2.205...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,2395,id_ff84602f7,GGAAAAUAGCAGAGGAAAUACUAGAGCAAUUGCAAAGGCCGAUCAU...,........((..((......))...)).........(((..........,EEEEEEEESSIISSHHHHHHSSIIISSXXXXXXXXXSSSHHHHHHH...,4.036,1,

In [7]:
?torch.utils.data.Dataset

Init signature: torch.utils.data.Dataset()
Docstring:     
An abstract class representing a :class:`Dataset`.

All datasets that represent a map from keys to data samples should subclass
it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
data sample for a given key. Subclasses could also optionally overwrite
:meth:`__len__`, which is expected to return the size of the dataset by many
:class:`~torch.utils.data.Sampler` implementations and the default options
of :class:`~torch.utils.data.DataLoader`. Subclasses could also
optionally implement :meth:`__getitems__`, for speedup batched samples
loading. This method accepts list of indices of samples of batch and returns
list of samples.

.. note::
  :class:`~torch.utils.data.DataLoader` by default constructs an index
  sampler that yields integral indices.  To make it work with a map-style
  dataset with non-integral indices/keys, a custom sampler must be provided.
File:           /Library/Frameworks/Python.frame

In [19]:
?torch.utils.data.Dataloader

Object `torch.utils.data.Dataloader` not found.


In [21]:
torch.__version__

'2.9.1'

In [49]:
dic = {"A":0, 'C':1, 'G':2, 'U':3}
def one_hot_enc(seq):
  ind = torch.tensor([dic[c] for c in seq])
  enc = F.one_hot(ind, num_classes=4)
  return enc.float()
  


In [ ]:
class RNADataset(Dataset):
    def __init__(self, df):
        # Filter to only rows that passed the SN filter (good quality data)
        df = df[df['SN_filter'] == 1.0].reset_index(drop=True)
        
        self.sequences = torch.stack([
            one_hot_enc(s) for s in df['sequence'].values
        ])
        self.sequences = self.sequences[:, 0:68, :]

        self.labels= torch.tensor(np.array(df['reactivity'].tolist(), dtype=np.float32))


    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return (
            self.sequences[idx],
            self.labels[idx]
        )

# Create dataset and dataloader
dataset = RNADataset(train_df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [77]:


class Model(nn.Module):
  def __init__(self, input_size=4, hidden_size=32, num_layers=2):
    super(Model, self).__init__()
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, device=device)
    self.fin = nn.Linear(hidden_size, out_features=1, device=device)
  
  def forward(self, x):
    # 32, 107, 4 x;.shape
    #assert x.shape == (32,68, 4), f"shape is not shaping, real: {x.shape}"
    x, _ = self.rnn(x)
    x = self.fin(x)
    x = x.squeeze(-1)
    return x

In [78]:
model = Model().to(device)
criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 20
train_losses = []


for ep in range(epochs):
  loss_e = 0
  model.train()
  for seq, reactivity in dataloader:
    seq = seq.to(device)
    reactivity= reactivity.to(device)
    output = model(seq)

    loss = criterion(output, reactivity)

    optim.zero_grad()

    loss.backward()

    optim.step()

    loss_e += loss.item()
  fin_los = loss_e / len(dataloader)
  print(ep, '\t',fin_los )
  train_losses.append(fin_los)


0 	 7.542316496372223
1 	 7.582425683736801
2 	 7.544573687016964
3 	 7.539677396416664
4 	 7.532070226967335
5 	 7.535928323864937
6 	 7.509113609790802
7 	 7.494763635098934
8 	 7.499777942895889
9 	 7.481428936123848
10 	 7.476079851388931
11 	 7.4727267771959305
12 	 7.48322057723999
13 	 7.482287362217903
14 	 7.470706418156624
15 	 7.482080854475498
16 	 7.448759533464909
17 	 7.458489008247852
18 	 7.434243246912956
19 	 7.422018297016621


In [47]:
dataset.sequences[0]

1589

In [ ]:
plt.plot(np.arange(len(train_losses)), train_losses)